优化算法:使用Keras提供的其他优化器，如梯度下降，看在其他算法下模型参数对模型训练和过拟合的速度有怎样的影响。
损失函数:尝试使用Keras其他可用的损失函数，探究选用其他的损失函数是否可以提升模型的性能。
学习率与迭代次数更新策略
更大的Batch Size:使用更大的Batch Size意味着模型在训练集和测试集上的数据操作规模更大了

In [ ]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.ion()   # interactive mode 画图不阻止程序运行
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications import *
from keras.applications.inception_v3 import preprocess_input
#from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers.normalization import BatchNormalization
from keras.layers import *
from keras.models import *

from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers



#from keras.utils import multi_gpu_model 

####################################################################
#设置可见的GPU数,注意不是并行训练的设置
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#看具体的模型参数设置在:https://www.tensorflow.org/api_docs/python/tf/keras/applications
MODELS = {"InceptionResNetV2":InceptionResNetV2}
#MODELS = {"DenseNet121":DenseNet121,"DenseNet201":DenseNet201,"InceptionResNetV2":InceptionResNetV2}
#"InceptionV3":InceptionV3,"DenseNet121":DenseNet121,
#       "DenseNet169":DenseNet169,"DenseNet201":DenseNet201,"Xception":Xception, 
#       "InceptionResNetV2":InceptionResNetV2,

#classes = ['pant_length_labels','coat_length_labels'] 
classes = ['collar_design_labels', 'neckline_design_labels', 'neck_design_labels']   
#记录各分类的最高验证率 
fai_result = []
#不同参数设置不同版本
version = "bestV1"
#version = ["bestV1","bestV2","bestV3","bestV4","bestV5"]
####################################################################

In [ ]:
for KEY, MODLE in MODELS.items():
    #为299*299,设置如下
    ppreprocess = preprocess_input
    if KEY in ["InceptionV3","Xception", "InceptionResNetV2"]:
        width = 299
    elif KEY == "NASNetLarge":
        width = 331
    else:
        width = 224
        ppreprocess = imagenet_utils.preprocess_input 
    print('######################在{0}下训练8个分类器####################'.format(KEY))
    #数据重读在for循环中有利于减小每次装入内存的数据量,切数据每次预处理会增加数据的多样性
    for cur_class in classes:
        print('#######{0}:{1}####################'.format(KEY,cur_class ))
        df_train = pd.read_csv('../train/Annotations/{0}.csv'.format(cur_class), header=None)
        #对载入内存副本的表,命名各series列的名称
        df_train.columns = ['image_id', 'class', 'label']
        df_load = df_train.copy()
        
        #当从train.csv抽取对应class的行时,需重置索引
        #默认为inplace=False,返回一个拷贝,原df_load不变,这里是直接改变原表的索引
        #默认下,重置索引会增加index索引列(其保存的是原有的索引列),所以可设置drop=Ture,或用如下方法删去原有索引
        df_load.reset_index(inplace=True)
        del df_load['index']
        
        n = len(df_load)
        n_class = len(df_load['label'][0])
        prefix_cls = cur_class.split('_')[0]
        print("选择的属性为:{0}, 种类的为:{1},样本数: {2}".format(cur_class, n_class, n))
        
        #图像尺度化,并读入内存,标签转化后也读入内存
        X = np.zeros((n, width, width, 3), dtype=np.uint8)
        y = np.zeros((n, n_class), dtype=np.uint8)
        for i in range(n):
            tmp_label = df_load['label'][i]
            if len(tmp_label) > n_class:
                print(df_load['image_id'][i])
            X[i] = cv2.resize(cv2.imread('../train/{0}'.format(df_load['image_id'][i])), (width, width))
            y[i][tmp_label.find('y')] = 1
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=60)
        print("数据装载到内存和数据分割完毕:{0},{1}".format(KEY,cur_class))
        
        #随机抽取内存图片,展示图片样例,2行4列
        #plt.figure(figsize=(12, 7))
        #for i in range(8):
            #random_index = random.randint(0, n-1)
            #plt.subplot(2, 4, i+1)
            #plt.imshow(X[random_index][:,:,::-1])
            #plt.title(y[random_index])
        #plt.savefig('../images/{0}/{0}_{1}_{2}.png'.format(prefix_cls, KEY, version),bbox_inches='tight')
        
        
        #定义模型
        #定义要finetune的模型结构
        cnn_model = MODLE(include_top=False, input_shape=(width, width, 3), weights='imagenet',pooling = 'avg')
        #输入模型网络的图片shape,如x = Input(shape=(256, 256, 3))
        inputs = Input((width, width, 3))#inputs = Input(shape=(784,))
        x = inputs
        #装入内存图片的预处理操作
        x = Lambda(ppreprocess, name='preprocessing')(x)
        #构建网络各模块逻辑连接
        x = cnn_model(x)
        x = BatchNormalization()(x)
        #x = AverragePooling2D(pool_size=(2, 2))(x)
        ######下面是新加的层########
        #因为设置了全局均值采样,所以没有flatten
        #x = Flatten(name='flatten')(x)#其他形式:model.add(Flatten()),out = Flatten()(x)
        x = Dense(1024, activation='relu',kernel_initializer=initializers.he_uniform(seed=None), name='fc1')(x)
        # n_class为对应属性的分类个数
        predictions = Dense(n_class, activation='softmax', name='softmax')(x)
        #将构建的模型网络实例化
        model = Model(inputs = inputs, outputs = predictions)
        
        #设置权参优化方法
        optimizer = SGD(lr=0.001, decay=0.1, momentum=0.9, nesterov=True)
        #optimizer = optimizers.Adam(lr=1e-4)
        #optimizer = Adam(lr=0.0001)
        #optimizer = 'rmsprop'
        
        #设置多GPU数据并行训练
        #多GPU训练,因为keras设计的自动保存最好模型,但是多GPU训练,其save()就没法用了,需定义各保存函数
        #model = multi_gpu_model(model, 2)  
        
        #所构建模型的编译,其中loss和metrics都可自定义,metrics=['accuracy',func自定义评价]
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Callback实现学习率调整方案和保存最好模型即EarlyStopping
        #schedule = Step([20], [1e-4, 1e-6])
        #history = model.fit(X_train, Y_train,
        #                    batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_test,Y_test),
        #                    callbacks=[schedule, keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')
        #当监测值不再改善时，该回调函数将中止训练.
        #当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练
        #keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

        #设置训练完之后,最好的模型保存路径,可设置监控准确率而不是监控loss的变化,如monitor='val_acc'
        #checkpointer = ModelCheckpoint(filepath='../models/{0}/{0}_{1}_{2}.best.h5'.format(prefix_cls, KEY, version), verbose=1, save_best_only=True)
        #训练开始,并保存训练过程的loss和acc变化,h代表history
        #reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,
          #                    patience=5, min_lr=0.001)
        h = model.fit(X_train, y_train, batch_size=10, epochs=36, 
                      #callbacks=[EarlyStopping(patience=22), checkpointer], 
                      #callbacks=[reduce_lr],
                      shuffle=True, 
                      validation_data=(X_valid,y_valid))
        #保存模型
        print("开始保存模型")
        model.save_weights('../models/{0}/{0}_{1}_{2}.best.h5'.format(prefix_cls, KEY, version))
        #是指取最后一个epochs训练得到的模型做一次整个验证集的准确率测试
        #score = model.evaluate(X_valid,y_valid,batch_size=32,verbose=0)
        #print ('{0}_{1}_{2}验证平均accuracy:{3}'.format(prefix_cls,KEY,version,score[1]))
        print("保存模型完成")
        #保存loss和acc的变化曲线
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(h.history['loss'])
        plt.plot(h.history['val_loss'])
        plt.legend(['loss', 'val_loss'])
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('{0}_{1}_loss'.format(prefix_cls, KEY))
        
        plt.subplot(1, 2, 2)
        plt.plot(h.history['acc'])
        plt.plot(h.history['val_acc'])
        plt.legend(['acc', 'val_acc'])
        plt.ylabel('acc')
        plt.xlabel('epoch')
        plt.title('{0}_{1}_accuracy'.format(prefix_cls, KEY))
        #保存损失和准确率变化的图像
        plt.savefig('../models/{0}/{0}_{1}_{2}.png'.format(prefix_cls, KEY,version),bbox_inches='tight')
        #显示图像
        plt.show()

        for valacc in h.history['val_acc']:
            if valacc >= 87:
                
                fai_result.append['{0}_{1}'.format(prefix_cls, KEY)]
        print(h.history['val_acc'])
        print(fai_result)
        
        #在测试集上预测并保存结果
        df_test = pd.read_csv('../test/Tests/question.csv', header=None)
        #定义各列名称
        df_test.columns = ['image_id', 'class', 'x']
        del df_test['x']
        df_load = df_test[(df_test['class'] == cur_class)].copy()
        df_load.reset_index(inplace=True)
        del df_load['index']
        
        n = len(df_load)
        X_test = np.zeros((n, width, width, 3), dtype=np.uint8)
        #读取测试集图像到内存并resize
        for i in range(n):
            X_test[i] = cv2.resize(cv2.imread('../test/{0}'.format(df_load['image_id'][i])), (width, width))
        #以batch_size处理的速度遍历测试集,处理并预测
        test_np = model.predict(X_test, batch_size=10)
        result = []
        for i, row in df_load.iterrows():
            tmp_list = test_np[i]
            tmp_result = ''
            for tmp_ret in tmp_list:
                tmp_result += '{:.6f};'.format(tmp_ret)
            #不要最后一个分号
            result.append(tmp_result[:-1])
        #预测结果导入内存表格的result列
        df_load['result'] = result     
        df_load.to_csv('../result/{1}/{0}_{1}_{2}.csv'.format(prefix_cls, KEY, version), header=None, index=False)